In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
BASE_DIR = "/kaggle/input/cassava-leaf-disease-classification/"
TRAIN_DIR = "/kaggle/input/cassava-leaf-disease-classification/train_images/"
TEST_DIR = "/kaggle/input/cassava-leaf-disease-classification/test_images/"

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import json
import cv2
from PIL import Image

In [ ]:
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    
print(json.dumps(map_classes, indent=2))

In [ ]:
label_list = [int(key) for key in map_classes.keys()]
label_list

In [ ]:
input_files = os.listdir(os.path.join(BASE_DIR, "train_images"))
print(f"Number of train images: {len(input_files)}")

In [ ]:
IMG_HEIGHT = 400
IMG_WIDTH = 400
batch_size = 32
PRE_TRAINED_MODEL = '../input/resnet50v02/Cassava_Best_ResNet50_Model_V02.hdf5'

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
test_filenames = os.listdir(TEST_DIR)
test_df = pd.DataFrame({
    'image_id': test_filenames
})
test_samples = test_df.shape[0]
test_samples

In [ ]:
import keras
import tensorflow as tf

keras.backend.clear_session() # clearing session
np.random.seed(42) # generating random see
tf.random.set_seed(42) # set.seed function helps reuse same set of random variables

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model(PRE_TRAINED_MODEL)
model.summary()

In [ ]:
#predict = model.predict(test_gen, steps=np.ceil(test_samples/batch_size))
#predict

In [ ]:
def get_augmented_images(image_id):
    image_path = os.path.join(TEST_DIR, image_id)
    image_data = Image.open(image_path)
    image_data = image_data.resize((IMG_HEIGHT, IMG_WIDTH), Image.ANTIALIAS)
    image_data = np.expand_dims(image_data, axis = 0)
    
    #print("Shape of Image :: {}".format(image_data.shape))
    
    test_datagen = ImageDataGenerator(validation_split = 0.2,
                                     rotation_range = 45,
                                     zoom_range = 0.4,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1)
    
    test_datagen = ImageDataGenerator(rescale=1.0/255, validation_split = 0.2,
                                     rotation_range = 45,
                                     zoom_range = 0.4,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1,
                                     featurewise_center = True,
                                     featurewise_std_normalization = True)
    
    test_datagen.fit(image_data)
    
    it = test_datagen.flow(image_data, batch_size=1)
    image_list = []
    image_list.append(image_data)
    for i in range(5):
        batch = it.next()
        #print("Shape of Augmented Image :: {}".format(batch.shape))
        image_list.append(batch)
    
    #print("Number of augmented images :: {}".format(len(image_list)))
    return image_list

In [ ]:
test_results_list = []
for image_id in test_df["image_id"]:
    image_list = get_augmented_images(image_id)
    
    predict_list = []
    for image_data in image_list:
        predict_class_1 = model.predict(image_data)
        print("Probabilities :: {}, :: Predicted Class : {}".format(predict_class_1[0],np.argmax(predict_class_1[0])))
        predict_list.append(predict_class_1[0])
    predict_array = np.array(predict_list)
    predict_class = np.mean(predict_array, axis=0)
    print("Augmented Probabilities :: {}, :: Augmented Predicted Class : {}".format(predict_class,np.argmax(predict_class)))
    test_results_dict = {}
    test_results_dict["image_id"] = image_id
    test_results_dict["label"] = np.argmax(predict_class)
    test_results_list.append(test_results_dict)

test_results_df = pd.DataFrame(test_results_list)
test_results_df.to_csv("submission.csv" ,index = False)


In [ ]:
submission = pd.read_csv("submission.csv")
submission.head(3)